In [1]:
#!/usr/bin/Rscript

suppressPackageStartupMessages(library(Rsamtools))
suppressPackageStartupMessages(library(GenomicAlignments))
suppressPackageStartupMessages(library(GenomicFiles))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(optparse))
suppressPackageStartupMessages(library(rtracklayer))
suppressPackageStartupMessages(library(GenomicRanges))
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(lattice))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library(foreach))
suppressPackageStartupMessages(library(doParallel))
suppressPackageStartupMessages(library(gridExtra))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(scales))
suppressPackageStartupMessages(library(fpc))
suppressPackageStartupMessages(library(bigWig))
registerDoParallel(cores=7);
source("./CoPRO_Functions.r");
genomef = "~/bin/genomes/hg19/hg19.2bit";
options(repr.plot.width=3, repr.plot.height=3, jupyter.plot_mimetypes = "image/svg+xml");

In [2]:
load('../data/CoPRO_hg19_dm6_paperfreeze/CoPRO_AllMerge_Pooled.Rdata');
Pooled = Pooled[ order(Pooled$ID5) ];
ReadTable = as.data.table(mcols(Pooled));
setkey(ReadTable, ID5);
# compute width & number of 3' ends
ReadTable[, width := abs(ID5-ID3)+1];

# count distinct 3' ends within first 60 bases
erly = ReadTable[ width >= 20 & width <= 60, .(PsPos=sum(C>0|R>0), tcap=sum(C), tunc=sum(U), ttot=sum(C+U+R+E)), by=ID5 ];

In [ ]:
ggplot( data=as.data.frame(erly[tcap>0,]), aes(x=PsPos, y=log2(tcap/min(tcap)))) + geom_hex(binwidth=c(1,0.5)) +
	scale_x_continuous(name = "Number of 3' ends", breaks=seq(0,40,5), limits=c(0,40)) +
 	scale_y_continuous(name = "Capped Reads (log2)", breaks=0:10, limits=c(0,10)) +
	scale_fill_gradientn(name = "TSN Count", colors=heatcols, trans='log10' ) +
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
		panel.background = element_blank(), legend.position='none',
		axis.line = element_line(colour = "black")
);

In [ ]:
N3p = hist(erly[PsPos<=30,PsPos], breaks=30, plot=F);
plot(x=N3p$mids, y=N3p$counts, log="y", xlim=c(0,30), ylim=c(1, 10^8), col="dodgerblue",
	main="TSB Calling Threshold", ylab="Number of TSBs", xlab="Number of 3' ends", type="l", lwd=2);
abline(v=5, col='gray');

In [ ]:
# start sites = at least 5 distinct 3' ends
erly = erly[ PsPos >= 5, ];
erly[, ElongR := 0 ];
ReadTable = ReadTable[ ID5 %in% erly$ID5, ];

In [ ]:
late = ReadTable[ width > 100 & width <= 400, .(tcap=sum(C), trpp=sum(R), tunc=sum(U), ttot=sum(C+U+R+E)), by=ID5 ];

CapR   = round( erly$tcap / (erly$tcap + erly$tunc), 2 );
FinCap = round( late$tcap / (late$tcap + late$tunc), 2 );
#erly[ ID5 %in% late$ID5, ElongR := round( late$ttot / (ttot + late$ttot), 2 ) ];
hist(CapR[CapR<1], xlim=c(0, 1), breaks=30);
hist(FinCap[FinCap<1], xlim=c(0, 1), breaks=30);
#hist(erly[ElongR>0, ElongR], xlim=c(0, 1), breaks=30);

In [ ]:
junk   =   late[ !is.na(FinCap) & FinCap <= 0.33, ID5 ];
starts =   erly[ !erly$ID5 %in% junk, ID5 ];
CapR   =   CapR[ erly$ID5 %in% starts ];
FinCap = FinCap[ erly$ID5 %in% starts ];
erly   =   erly[ erly$ID5 %in% starts ];
TSB    = Pooled[ Pooled$ID5 %in% starts ];

In [ ]:
rm(ReadTable, late);
invisible(gc());

In [ ]:
base5 = Pooled;
base5 = resize(base5, width=1, fix='start');
rm(Pooled);
invisible(gc());

In [ ]:
# sum over 3' ends to simplify TSBs
sTSB = TSB[ !duplicated(TSB$ID5) ];
# collapse onto 5' base
sTSB = resize( sTSB, width=1 );
sTSB$ID3 = NULL;
msgout(pn(length(sTSB)), "TSB calls (at least 5 unique 3' ends, reads >100 nt at least 33% capped)");

In [ ]:
sTSB$C = aggregate( C ~ ID5, data=TSB, FUN=sum )[,2];
sTSB$U = aggregate( U ~ ID5, data=TSB, FUN=sum )[,2];
sTSB$R = aggregate( R ~ ID5, data=TSB, FUN=sum )[,2];
sTSB$E = aggregate( E ~ ID5, data=TSB, FUN=sum )[,2];

In [ ]:
sTSB$CapR = CapR;
sTSB$FinCap = FinCap;
#sTSB$ElongR = round(erly$ElongR, 2);
hist(100*sTSB$CapR[sTSB$C>3&sTSB$CapR<1], breaks=50, xlim=c(0, 100), xlab="% Capped", ylab="Number of TSBs", col=tblue, main="Cap Ratios (filt.)")

Ratio = ifelse(sTSB$C>sTSB$R, sTSB$R/sTSB$C, sTSB$C/sTSB$R);
hist(Ratio, breaks=80, xlim=c(0, 1), ylim=c(0, 5E4), main="Cap / RppH", col=tblue);

In [ ]:
# remove sites with more than 5-fold difference in Capped and RppH samples
junk = Ratio <= 0.2;
msgout(pn(sum(junk,na.rm=T)), "TSBs have >5x disagreement between Cap and RppH");

sTSB = sTSB[!junk & seqnames(sTSB) != 'chrM'];
TSB  = TSB[TSB$ID5 %in% sTSB$ID5];

In [ ]:
# Define TSS as clusters of TSB within 60 bases of each other
TSS = compute_TSS( sTSB, 60 );
msgout(pn(length(TSS)), "TSS calls");

# assign each TSB to appropriate TSS
hits = findOverlaps( sTSB, TSS, type="any" );
sTSB$TSS = TSS$ID5[ hits@to ];
 TSB$TSS = sTSB$TSS[ match(TSB$ID5, sTSB$ID5) ];
# count TSBs within each cluster
TSS$nTSB = plyr::count(hits@to)$freq; # get frequency of TSB hits

# count capped and uncapped reads in each TSS
hits = findOverlaps( base5, TSS );
TSS$Cap   = aggregate( base5$C[hits@from] ~ hits@to, FUN=sum )[,2];
TSS$Uncap = aggregate( base5$U[hits@from] ~ hits@to, FUN=sum )[,2];
TSS$RppH  = aggregate( base5$R[hits@from] ~ hits@to, FUN=sum )[,2];

In [ ]:
# compute termination ratios from max TSB in each TSS
maxTSS = as.data.table( mcols(sTSB) );
# find max capped TSB in each TSS
maxTSS[, isMax := (which.max(C) == seq_len(.N)), by=TSS ];
TW = sTSB[ maxTSS[,isMax] ];
# shift 10 bp downstream
TW = GenomicRanges::shift(TW, 10);
# expand 90 bp downstream
TW = resize(TW, 90);
# get all reads starting in these regions
hits = findOverlaps( base5, TW );
HH = unique(hits@to);
TW$Cap = 0;
TW$Uncap = 0;
TW$RppH = 0;
TW$Cap[HH]   = aggregate( base5$C[hits@from] ~ hits@to, FUN=sum )[,2];
TW$Uncap[HH] = aggregate( base5$U[hits@from] ~ hits@to, FUN=sum )[,2];
TW$RppH[HH]  = aggregate( base5$R[hits@from] ~ hits@to, FUN=sum )[,2];
# add a psuedocount to prevent dividing by 0
TSS$TermR = round( TW$Uncap / (TW$Cap + TW$Uncap + 0.01), 2);

In [ ]:
msgout(pn(sum(TSS$Cap <= 0.8 & width(TSS) < 5)), "TSS with <5 Capped reads and <5 bp");
TSS = TSS[ TSS$Cap > 0.8 & width(TSS) >= 5 ];
msgout(pn(sum(!sTSB$TSS %in% TSS$ID5)), "sTSBs removed");
sTSB = sTSB[ sTSB$TSS %in% TSS$ID5 ];
 TSB =  TSB[  TSB$TSS %in% TSS$ID5 ];


In [ ]:
wsizes = c( seq(2, 20, 2), seq(25, 1000, 25), seq(1200, 5000, 200), seq(6E3, 110E3, 4E3) );
TIDct = wsizes;
for( i in 1:length(wsizes) ) {
	TIDct[i] = length(compute_TREs( sTSB, wsizes[i] ))/1000;
}

In [ ]:
Clustering = data.frame( Distance=wsizes, Count=TIDct );
allmod = lm( log(Count) ~ log(Distance), data=Clustering, subset=wsizes<=5E3 );
summary( allmod );
predall = exp( predict(allmod, list(Distance=wsizes)) );

In [ ]:
CFits = Clustering[3:length(wsizes) -1,];
colnames(CFits) = c('A', 'B');
for( i in 3:length(wsizes) -1 ) {
    if(wsizes[i] >= 4000) next;
    sgmod = lm( log(Count) ~ log(Distance), data=Clustering, subset=Distance <= wsizes[i] );
    predsg = exp( predict(sgmod, list(Distance=wsizes)) );
    Residual = Clustering;
    Residual$Count = Residual$Count - predsg;
    Residual$Count[Residual$Count < 1] = 1;
    bgmod = lm( log(Count) ~ log(Distance), data=Clustering, subset=Distance >= wsizes[i] & Distance<=4E3 );
    CFits[i-1,1] = round(summary(sgmod)$adj.r.squared,3)
    CFits[i-1,2] = round(summary(bgmod)$adj.r.squared,3);
}
sgmod = lm( log(Count) ~ log(Distance), data=Clustering, subset=Distance <= 500 );
bgmod = lm( log(Count) ~ log(Distance), data=Clustering, subset=Distance >  1000 & Distance <= 5E3 );
predbg = exp( predict(bgmod, list(Distance=wsizes)) );
predsg = exp( predict(sgmod, list(Distance=wsizes)) );
#print(cbind(wsizes[3:length(wsizes)-1], CFits$A, CFits$B));

In [ ]:
pdf("../plots/PhaseTransition.pdf", width=5, height=5);
par(mar=c(4,4,1,1)+0.5);

matplot(
	x=wsizes/1000, y=cbind(TIDct, predbg, predsg, predall), type='l', lwd=2, lty=c(1,2,2,2),
	xlab='Clustering Distance (kbp)', ylab='Thousands of Clusters', log='xy',
	col=c("dodgerblue", 'black', 'black', 'gray'), xlim=c(0.1, 12), ylim=c(10, 70)
);
legend( 'topright', legend=c('CoPRO', 'One exp.', 'Two exps.'), col=c('dodgerblue', 'gray', 'black'), lty=c(1,2,2));

matplot(
	x=wsizes[3:length(wsizes)-1]/1000, y=CFits, type='l', lwd=2, lty=1,
	xlab='Clustering Distance (kbp)', ylab=expression('R'^2), main='Best Fit',
	xlim=c(0, 5), ylim=c(0.6, 1), col=c("dodgerblue", 'firebrick')
);
abline(v=c(462), col='gray', lty=3);
legend( 'bottomright', legend=c('Fast', 'Slow'), col=c('dodgerblue', 'firebrick'), lty=1);

matplot(
	x=wsizes/1000, y=cbind(TIDct, predbg, predsg), type='l', lwd=2, lty=c(1,2,2,2),
	xlab='Clustering Distance (kbp)', ylab='Thousands of TIDs', log='xy',
	xlim=c(0.1, 12), ylim=c(15, 60), col=c("dodgerblue", 'black', 'black', 'gray')
);
legend( 'topright', legend=c('CoPRO', 'Best Fits'), col=c('dodgerblue', 'black'), lty=c(1,2,2));

dev.off();

In [ ]:
# Cluster TSSs into TREs
TREs = compute_TREs( sTSB, 475 );
msgout(pn(length(TREs)), "TREs");

# count number of TSS on each strand
pl = strand(TSS) == "+";
TREs$pTSS = countOverlaps( TREs, TSS[pl] );
TREs$nTSS = countOverlaps( TREs, TSS[!pl] );
# Add TRE ID to each TSS
hits = findOverlaps( TSS, TREs );
TSS$TREID = NA;
TSS$TREID[hits@from] = TREs[hits@to]$ID5;
TREcap = aggregate(Cap ~ TREID, data=TSS, FUN=sum);
TREs$Cap = TREcap[match(TREs$ID5, TREcap$TREID),2];

In [ ]:
ggplot( data=as.data.frame(mcols(TREs)), aes( x=width(TREs), y=Cap )) + geom_hex(binwidth=c(0.25,0.25)) +
	scale_x_continuous(name = "TID Width (bp)", breaks=2^seq(4,11,2), limits=c(16,2^11), trans='log2') +
 	scale_y_continuous(name = "Capped Reads", breaks=2^seq(0,10,2), limits=c(1,2^10), trans='log2') +
	scale_fill_gradientn(name = "TID Count", colors=heatcols, trans='log10' ) +
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
		panel.background = element_blank(), legend.position='none',
		axis.line = element_line(colour = "black")
);

In [ ]:
maxTSS = as.data.table( mcols(TSS) );
# determine which site has max capped reads at each TRE
maxTSS[, isMax := (which.max(Cap) == seq_len(.N)), by=TREID ];
maxID5 = maxTSS[isMax == T, ID5];
mTSS = TSS[ TSS$ID5 %in% maxID5 ];
TREs$mTSS = mTSS$ID5[match(TREs$ID5, mTSS$TREID)];

In [ ]:
TREavgTerm = aggregate( TSS$TermR ~ TSS$TREID, FUN=mean );
hist(100*TREavgTerm[,2], breaks=50, xlim=c(0, 100), xlab="% Terminating", ylab="Number of TREs", col=tblue)
highTerm = TREavgTerm[ TREavgTerm[,2] >= 0.90, 1 ];

# remove single-stranded TREs with high avg termination
keep = (!TREs$ID5 %in% highTerm) | (TREs$pTSS & TREs$nTSS);
TREs = TREs[ keep & TREs$Cap > 2 ];
TSS = TSS[ TSS$TREID %in% TREs$ID5 ];
sTSB = sTSB[ sTSB$TSS %in% TSS$ID5 ];
TSB = TSB[ TSB$ID5 %in% sTSB$ID5 ];

msgout(pn(sum(!keep)), "TREs removed by secondary filters");
msgout(pn(length(TREs)), "TREs");
msgout(pn(length(TSS )), "TSSs");
msgout(pn(length(sTSB)), "TSBs");
hist(100*TSS$TermR, breaks=50, xlim=c(0, 100), xlab="% Terminating", ylab="Number of TSS", col=tblue)

In [ ]:
# read in CAGE data
CAGE = import.bw("../data/K562_paCAGE_pl.bw"); # reprocessed for 5'base resolution
CAGEmn = import.bw("../data/K562_paCAGE_mn.bw"); # reprocessed for 5'base resolution
strand(CAGE) = "+";
strand(CAGEmn) = "-";
CAGE = append(CAGE, CAGEmn);
rm(CAGEmn);
seqlengths(CAGE) = seqlengths(sTSB)[names(seqlengths(CAGE))]
hits = findOverlaps( sTSB, CAGE );
sTSB$CAGE = as.integer(0);
sTSB$CAGE[hits@from] = abs(CAGE$score[hits@to]);

In [ ]:
# compute stability for individual TSBs
sTSB$Stability = factor('Unstable', levels=c('Unstable', 'Stable'))
# Stable if at least 8 CAGE reads
sTSB$Stability[ sTSB$CAGE >= 8 ] = "Stable";
msgout('TSBs')
summary(sTSB$Stability)

In [ ]:
TSS$Stability = factor(ifelse( countOverlaps(TSS, sTSB[sTSB$Stability=='Stable'])>0, 'Stable', 'Unstable' ));
msgout('TSSs')
summary(TSS$Stability)

In [ ]:
#sTSB$CAGE = NULL;
# compute number of stable and unstable TSS in each TRE
TREs$nS = countOverlaps(TREs, TSS[TSS$Stability == 'Stable']);
TREs$nU = countOverlaps(TREs, TSS[TSS$Stability == 'Unstable']);

In [ ]:
# Read transcription unit info
TX = as(read.table("../data/TranscriptionUnits.bed", header=T, sep="\t"), "GRanges");
nTUs = length(TX);
# collapse to TSS
TXtss = resize(TX, 1);
# expand 500 bp in both directions
TXtss = resize(TXtss, 1000, fix="center");

In [ ]:
# Classify TREs relative to TUs (independent of expression)
TREs$Location = factor("Intergenic", levels=c("Promoter", "Intergenic", "Intragenic"));
TMap = findOverlaps( TREs, TX );
intergenic = TREs[ TMap@from ];
TREs$Location[ TMap@from ] = "Intragenic";
TMap = findOverlaps( TREs, TXtss );
TREs$Location[ TMap@from ] = "Promoter";
promoters = TREs[ TMap@from ];
intergenic = intergenic[ !intergenic %in% promoters ];
other = TREs[ -TMap@from ];
other = other[ !other %in% intergenic ];
# Assign TSS/TSB as well
TSS$Location = TREs$Location[ match(TSS$TREID, TREs$ID5) ];
sTSB$Location = TSS$Location[ match(sTSB$TSS, TSS$ID5) ];

In [ ]:
# Annotate transcribed TUs
TMap = findOverlaps( TSS, TXtss );
TX = TX[ unique(TMap@to) ];
TXtss = TXtss[ unique(TMap@to) ];
TX$nTSS = plyr::count(TMap@to)$freq;
TMap = findOverlaps( TSS, TXtss );
# assign all TSS to nearby TU
TSS$TUID = NA;
TSS$TUID[ TMap@from ] = TXtss$ID[TMap@to];
TSS$TUname = NA;
TSS$TUname[ TMap@from ] = as.character(TX[TMap@to]$name);

In [ ]:
# many TUs have more than 1 matched TSS, so choose max TSS
tmax = aggregate( TSS[TMap@from]$Cap ~ TMap@to, FUN=max );
colnames(tmax) = c('to', 'Cap');
x = match( TMap@to, tmax$to );
# select max bases
TMap = TMap[ TSS$Cap[TMap@from] == tmax$Cap[x] ];
# remove any bases tied for max
TMap = TMap[ !duplicated(TMap@to) ];
msgout(pn(length(TMap@to)), "active transcription units")
msgout(round(100*length(TMap@to) / nTUs), "% of all annotated TUs");
TX$maxTSS = TMap@from;
TX$TREID = TSS$TREID[ TX$maxTSS ];
TX$TermR = round( TSS$TermR[ TX$maxTSS ], 2);

In [ ]:
# check basic stats for our TSS and TRE calls
TSSh = hist(TSS$nTSB, breaks=max(TSS$nTSB), plot=F);
plot(x=TSSh$mids, y=TSSh$counts, log="y", xlim=c(0,60), col="steelblue4", lwd=2,
	main="Start Bases per TSS", ylab="Number of TSS", xlab="Number of TSBs", type="l");


In [ ]:
TSSw = hist(width(TSS), breaks=max(width(TSS)), plot=F);
plot(x=TSSw$mids, y=TSSw$counts, log="y", xlim=c(0,150), col="steelblue4",
	main="TSS Sizes", ylab="Number of TSS", xlab="TSS Width", type="l", lwd=2);

In [ ]:
TREw = hist(width(TREs), breaks=300, plot=F);
plot(x=TREw$mids, y=TREw$counts, log="y", xlim=c(0,400), col="steelblue4",
	main="TRE Sizes", ylab="Number of TREs", xlab="TRE Width", type="l", lwd=2);
#abline(v=30, col='gray');

In [ ]:
#msgout(pn(sum(width(TREs)<30)), "TREs <30 bp removed")
# remove small/lonely TREs
#TREs = TREs[width(TREs)>=30];

In [ ]:
# num TSS on each strand
ggplot( data=as.data.frame(TREs), aes(x=1+pTSS, y=1+nTSS)) + stat_bin2d(binwidth=1) +
	scale_x_discrete(name = "Number of Forward TSS", limits=0:7, labels=-1:6) +
 	scale_y_discrete(name = "Number of Reverse TSS", limits=0:7, labels=-1:6) +
	scale_fill_gradientn(name = "TSS Count", colors=heatcols ) +
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
		panel.background = element_blank(), legend.position=c(0.8, 0.25),
		axis.line = element_line(colour = "black")
);

In [ ]:
# num stable and unstable TSS
ggplot( data=as.data.frame(TREs), aes(x=1+nS, y=1+nU)) + stat_bin2d(binwidth=1) +
	scale_x_discrete(name = "Number of Stable TSS", limits=0:11, labels=-1:8) +
 	scale_y_discrete(name = "Number of Unstable TSS", limits=0:11, labels=-1:8) +
	scale_fill_gradientn(name = "TSS Count", colors=heatcols, trans='log10', limits=c(1, 2E4) ) +
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
		panel.background = element_blank(), legend.position=c(0.8, 0.4),
		axis.line = element_line(colour = "black")
);

In [ ]:
# remove chrY & load bigWig data
TSB  =  TSB[ seqnames( TSB) != 'chrY' ];
TSS  =  TSS[ seqnames( TSS) != 'chrY' ];
sTSB = sTSB[ seqnames(sTSB) != 'chrY' ];
TREs = TREs[ seqnames(TREs) != 'chrY' ];
#CTCFbw = load.bigWig("../data/K562_CTCF.bw");
#TBPbw = load.bigWig("../data/K562_TBP.bw");
#TSS$CTCF = region.bpQuery.bigWig(CTCFbw, as.character(seqnames(TSS)), start(TSS) - 100, end(TSS) + 100, op="max");
#TSS$TBP  = region.bpQuery.bigWig( TBPbw, as.character(seqnames(TSS)), start(TSS) - 100, end(TSS) + 100, op="max");


In [ ]:
# compare TSS properties at promoters vs elsewhere
wTSS = TSS[width(TSS)>5];
intergenic = wTSS[ !is.na(wTSS$Location) & wTSS$Location == "Intergenic" ];
intragenic = wTSS[ !is.na(wTSS$Location) & wTSS$Location == "Intragenic" ];
promoters  = wTSS[ !is.na(wTSS$Location) & wTSS$Location == "Promoter"   ];
hist( promoters$TermR, breaks=seq(0,1,by=0.05), xlim=c(0,1), ylim=c(0,3500),
	col="steelblue3", xlab='Termination Ratio', main="Promoter TSS"
);
hist( intergenic$TermR, breaks=seq(0,1,by=0.05), xlim=c(0,1), ylim=c(0,3500),
	col="green4", xlab='Termination Ratio', main=paste0("Intergenic TSS")
);
hist( intragenic$TermR, breaks=seq(0,1,by=0.05), xlim=c(0,1), ylim=c(0,3500),
	col="firebrick", xlab='Termination Ratio', main="Intragenic TSS"
);

In [ ]:
# save results
tline = c('track name="TSB" itemRgb="On"', rep('', 8));
strcol = ifelse(strand(sTSB) == "+", '200,0,0', '0,0,170');
out = cbind(as.character(seqnames(sTSB)), start(sTSB), end(sTSB), substr(as.character(sTSB$Stability), 1, 1), sTSB$CapR, as.character(strand(sTSB)), start(sTSB), end(sTSB), strcol);
out = rbind(tline, out);
write.table(out, "../data/K562_TSBs.bed", row.names=F, col.names=F, quote=F)

strcol = ifelse(strand(TSS) == "+", '200,0,0', '0,0,170');
tline = c('track name="TSS" itemRgb="On"', rep('', 8));
out = cbind(as.character(seqnames(TSS)), start(TSS), end(TSS)+1, substr(as.character(TSS$Stability), 1, 1), TSS$TermR, as.character(strand(TSS)), start(TSS), end(TSS), strcol);
out = rbind(tline, out);
write.table(out, "../data/K562_TSS.bed", row.names=F, col.names=F, quote=F)

out = cbind(as.character(seqnames(TREs)), start(TREs), end(TREs)+1, as.character(TREs$Stability), TREs$TermR, as.character(strand(TREs)));
write.table(out, "../data/K562_TREs.bed", row.names=F, col.names=F, quote=F);

In [ ]:
maxTSS = as.data.table( mcols(sTSB) );
# determine which site has max capped/rpph reads at each TSS
maxTSS[, isMax := (which.max(C) == seq_len(.N)), by=TSS ];
mTSB = sTSB[ maxTSS[,isMax] ];
mTSB = mTSB[ order(mTSB$ID5) ];

In [ ]:
hits = findOverlaps( mTSB, TSS );
TSS$mTSB[hits@to] = mTSB[hits@from]$ID5;

In [ ]:
# get flanking RNA sequences from genome
flank5 = GenomicRanges::promoters(mTSB[mTSB$C>2], upstream=5, downstream=6);
# NOTE: import.2bit ignores strand, so we must manually complement the sequence.
seq5 = import.2bit(genomef, which=flank5);
seq5[ strand(flank5) == '-' ] = reverseComplement(seq5[ strand(flank5) == '-' ]);
save(seq5, file="../data/K562_mTSB_seq5context.Rdata")

In [ ]:
export(seq5, "../out/K562_mTSB_seq5context.fasta.gz")

In [ ]:
# divergent pairings
# find maxTSB on - strand within 300 bp of maxTSB on + strand
plTSB  = GenomicRanges::promoters( mTSB[ strand(mTSB) == "+" ], upstream=300, downstream=0 );
mnTSB  = mTSB[ strand(mTSB) == "-" ];
hits   = findOverlaps( plTSB, mnTSB, ignore.strand=T );
# max ID5 on - strand = closest to + TSS
pairs  = aggregate( mnTSB$ID5[hits@to] ~ plTSB[hits@from]$ID5, FUN=max );

In [ ]:
TSS$Pair = NA;
# map maxTSBs to their TSS
plTSS = mTSB$TSS[match(pairs[,1], mTSB$ID5)];
mnTSS = mTSB$TSS[match(pairs[,2], mTSB$ID5)];
# save pairing in TSS
TSS$Pair[ match(plTSS, TSS$ID5) ] = mnTSS;
TSS$Pair[ match(mnTSS, TSS$ID5) ] = plTSS;

In [ ]:
# write bed file for paired TSS
Paired = TSS[!is.na(TSS$Pair)]
DivPairs = data.frame( Chr = seqnames(Paired),
                       Start = start(Paired)-1, End = end(Paired),
                       names = Paired$TUname, Cap = Paired$Cap,
                       Strand = strand(Paired), Pair = Paired$Pair, mTSB = Paired$mTSB)
DivPairs = DivPairs[order(DivPairs$Pair,-DivPairs$Cap),]
DivPairs = DivPairs[!duplicated(DivPairs$Pair),]
DivPairs = DivPairs[order(abs(DivPairs$mTSB)),]
strcol = ifelse(DivPairs$Strand == "+", '200,0,0', '0,0,170');
DivPairs = as.matrix(DivPairs);
PairBed = cbind( DivPairs[,1:6], DivPairs[,2:3], strcol );
tline = c('track name="TSS" itemRgb="On"', rep('', 8));
PairBed = rbind( tline, PairBed );
write.table(PairBed, file='../data/K562_DivergentPairs.bed', quote=F, row.names = F, col.names = F);

In [ ]:
save(TSB, file="../data/K562_CoPRO_TSB.Rdata");
save(sTSB, file="../data/K562_CoPRO_simpleTSB.Rdata");
save(TSS, file="../data/K562_CoPRO_TSS.Rdata");
save(TREs, file="../data/K562_CoPRO_TREs.Rdata");
save(TX, file="../data/K562_CoPRO_TX.Rdata");